In [123]:
import os
import re
import subprocess
import shutil
from datetime import datetime
from typing import List

import requests
import tarfile
from bs4 import BeautifulSoup

def math_scraper(input_names: List[str], output_name: str) -> None:
    """
    Reads the input LaTeX file and extracts all equations, align, and $$ environments
    into the output file, with each math environment on a separate line.

    Args:
        input_names (List[str]): A list of the names of the input LaTeX files.
        output_name (str): The name of the output text file.

    Returns:
        None

    Example Usage:
        >>> math_scraper(['input1.tex', 'input2.tex'], 'output.txt')
    """
    # Open the output file for writing
    with open(output_name, 'w') as output_file:

        # Define regular expressions for equation, align, and $$ environments
        equation_re = re.compile(r'\\begin{equation\*?}(.*?)\\end{equation\*?}', re.DOTALL)
        align_re = re.compile(r'\\begin{align\*?}(.*?)\\end{align\*?}', re.DOTALL)
        dollar_re = re.compile(r'\\begin{equation\*?}(.*?)\\end{equation\*?}|'
                               r'\\begin{align\*?}(.*?)\\end{align\*?}|'
                               r'\$\$(.*?)\$\$', re.DOTALL)

        # Find matches within each input file and write them to the output file
        for input_name in input_names:
            with open(input_name, 'r') as input_file:
                input_contents = input_file.read()
                for match in equation_re.findall(input_contents):
                    output_file.write(match.strip() + '\n\n')

                for match in align_re.findall(input_contents):
                    output_file.write(match.strip() + '\n\n')

                for match in dollar_re.findall(input_contents):
                    output_file.write(match[1].strip() + '\n\n')

        input_file.close()
    output_file.close()



In [116]:
def file_scraper(directory_name: str, output_name: str) -> None:
    """
    Reads all files in the specified directory and writes their contents
    to the specified output file, with each file's contents separated by a newline.

    Args:
        directory_name (str): The name of the directory containing the input files.
        output_name (str): The name of the output text file.

    Returns:
        None

    Example Usage:
        >>> file_scraper('my_directory', 'output.txt')
    """
    # Create a list of all files in the directory
    file_list = [f for f in os.listdir(directory_name) if os.path.isfile(os.path.join(directory_name, f))]

    # Check if the output file already exists in the parent directory
    parent_directory = os.path.dirname(directory_name)
    file_path = os.path.join(parent_directory, output_name)
    if os.path.isfile(file_path):
        # Add a suffix to the output file name if it already exists
        filename, extension = os.path.splitext(output_name)
        i = 1
        while os.path.isfile(os.path.join(parent_directory, f"{filename}_{i}{extension}")):
            i += 1
        output_name = f"{filename}_{i}{extension}"

    # Open the output file for writing
    with open(os.path.join(parent_directory, output_name), 'w') as output_file:
        # Write the contents of each file to the output file
        for file_name in file_list:
            with open(os.path.join(directory_name, file_name), 'r') as input_file:
                output_file.write(input_file.read().strip() + '\n')


In [120]:
def unpack_tex_files(download_directory: str) -> None:
    """
    Deletes all files in the specified directory that are not '.tex' files or compressed '.pdf' files containing '.tex' files.
    Extracts all '.tex' files from compressed '.tar.gz' or '.tar' files to the specified directory.

    Args:
        download_directory: The path of the directory containing the downloaded files.

    Raises:
        subprocess.CalledProcessError: If the 'file' command fails to execute properly.

    """
    file_names = os.listdir(path=download_directory)

    for idx, filename in enumerate(file_names):
        if filename[-4:] != ".tex":
            print(f" ===== Handling file nr: {idx+1} with id: {filename} =====")
            full_filename = os.path.join(download_directory, filename)
            file_type_info_list = subprocess.check_output(["file", full_filename]).decode().strip().split()
            file_removed = False

            # If file_type_info_list contains 'PDF', file is either a '.pdf' or a compressed '.pdf' file.
            # We want only '.tex' files, so we delete it.
            if "PDF" in file_type_info_list:
                print(f" Deleting file nr: {idx+1} with id: {filename} - no '.tex' file found...")
                file_removed = True
                os.remove(full_filename)

            # If file_type_info_list contains 'gzip' and 'compressed', file is a compressed '.tar.gz' file.
            # We extract all '.tex' files to the specified directory and delete the original file.
            elif "gzip" in file_type_info_list and "compressed" in file_type_info_list:
                try:
                    tar = tarfile.open(full_filename, 'r:gz')
                    file_types = [os.path.splitext(os.path.basename(member.name))[1] for member in tar.getmembers()]
                    tar.close()
                    if ".tex" not in file_types:
                        print(f" Deleting file nr: {idx+1} with id: {filename} - no '.tex' file found...")
                        file_removed = True
                        os.remove(full_filename)
                    else:
                        tar = tarfile.open(full_filename, 'r:gz')
                        tar.extractall(path=download_directory, members=[m for m in tar.getmembers() if os.path.splitext(m.name)[1] == ".tex"])
                        tar.close()
                except tarfile.TarError as e:
                    print(f"{e} - file nr: {idx+1} with id: {filename} skipped an deleted...")
                    os.remove(full_filename)

            # If file_type_info_list contains 'tar', 'POSIX' and 'archive', file is a '.tar' file.
            # We extract all '.tex' files to the specified directory and delete the original file.
            elif "tar" in file_type_info_list and "POSIX" in file_type_info_list and "archive" in file_type_info_list:
                try:
                    tar = tarfile.open(full_filename, 'r:')
                    file_types = [os.path.splitext(os.path.basename(member.name))[1] for member in tar.getmembers()]
                    tar.close()
                    if ".tex" not in file_types:
                        print(f" Deleting file nr: {idx+1} with id: {filename} - no '.tex' file found...")
                        file_removed = True
                        os.remove(full_filename)
                    else:
                        tar = tarfile.open(full_filename, 'r:')
                        tar.extractall(path=download_directory, members=[m for m in tar.getmembers() if os.path.splitext(m.name)[1] == ".tex"])
                        tar.close()
                except tarfile.TarError as e:
                    print(f"{e} - file nr: {idx+1} with id: {filename} skipped an deleted...")
                    os.remove(full_filename)
            # Always remove original after.
            if not file_removed:
                os.remove(full_filename)

    # Move files from created sub-folders to parent folder
    move_files_to_parent(download_directory=download_directory)

In [121]:
def download_arxiv_papers(category: str, num_papers: int, download_dir: str) -> None:
    """
    Downloads STEM (Science, Technology, Engineering and Math) category arXiv papers in the specified category and saves them to the specified directory.

    Parameters:
        category (str): The arXiv category to download papers from.
        num_papers (int): The number of papers to download.
        download_dir (str): The directory to save the downloaded papers to.

    Returns:
        None.
    """
    # Physics, Mathematics, Computer Science, Quantitative biology, Electrical Engineering and Systems Science
    # Quantitative finance, Economics, Statistics
    STEM = ['physics','math', 'cs', 'eess', 'q-bio', 'q-fin', 'econ', 'stat']
    if category not in STEM:
        raise Exception(f'Category: {category} is not in the defined STEM categories: {STEM}')

    # Create the download directory if it doesn't exist
    if not os.path.exists(download_dir):
        # Notify the user that the download directory was created
        print(f"N.B.: Path '{download_dir}' didn't already exist, so it was created... ")
        os.makedirs(download_dir)

    # Request the arXiv category page and parse it with BeautifulSoup
    # Get the current date and time
    now = datetime.now()
    # Format the date as YY/MM
    current_formatted_date = now.strftime("%y%m")

    # Construct the URL of the arXiv category page for the current date and specified category
    url = f"https://arxiv.org/list/{category}/{current_formatted_date}"
    # Send an HTTP request to the URL and get the response
    response_1 = requests.get(url)
    # Parse the HTML content of the response with BeautifulSoup
    soup = BeautifulSoup(response_1.content, 'html.parser')
    # Construct a regular expression that matches links to the arXiv category page for the current date and specified category
    regex = re.compile(re.escape(f'/list/{category}/{current_formatted_date}'))
    # Find all links on the page that match the regular expression
    links = soup.find_all('a', href=regex)
    link = None
    # Loop over the links and find the link with the text ">all</a>"
    for l in soup.find_all('a', href=regex):
        if str(l)[-8:] == ">all</a>":
            link = l
    # Send an HTTP request to the URL of the page that contains links to individual papers in the category
    response_2 = requests.get(url+link['href'])
    # Parse the HTML content of the response with BeautifulSoup
    soup = BeautifulSoup(response_2.content, 'html.parser')

    # Extract the links to the individual papers from the page
    paper_links = soup.find_all('a', title='Abstract')

    # Raise an exception if the number of papers requested is greater than the number of papers available
    if num_papers > len(paper_links):
        raise Exception(f"{num_papers} papers requested, but only: {len(paper_links)} available in category: '{category}' in YY/MM: {current_formatted_date} ...")
    else:
        # Notify the user of the number of papers being downloaded
        print(f"======= Downloading: {num_papers} (out of {len(paper_links)}) papers in category: '{category}' in YY/MM: {current_formatted_date} =======")

    # Loop over the paper links and download the source files for each paper
    for i, link in enumerate(paper_links):
        if i >= num_papers:
            break

        # Get the URL of the paper page
        paper_url = "https://arxiv.org" + link['href']

        # Extract the paper ID from the URL
        paper_id = paper_url.split('/')[-1]

        # Construct the URL of the source files
        source_url = f"https://arxiv.org/e-print/{paper_id}"

        print("Downloading paper: ", i+1, " with id: ", paper_id, " ...")
        # Download the source files and save them to the download directory
        response_3 = requests.get(source_url)

        with open(download_dir+str(paper_id), "wb") as f:
            f.write(response_3.content)
        f.close()


In [119]:
nr_papers = 50
STEM_category = 'physics'
download_directory = "arxiv_papers/"
download_arxiv_papers(STEM_category, nr_papers, download_dir=download_directory)

======= Downloading: 50 (out of 1360) papers in category: 'physics' in YY/MM: 2303 =======


In [122]:
unpack_tex_files(download_directory=download_directory)

 ===== Handling file nr: 1 with id: 2303.00361 =====
 Deleting file nr: 1 with id: 2303.00361 - no '.tex' file found...
 ===== Handling file nr: 2 with id: 2303.00395 =====
 ===== Handling file nr: 3 with id: 2303.00197 =====
 ===== Handling file nr: 4 with id: 2303.00622 =====
 ===== Handling file nr: 5 with id: 2303.00248 =====
 Deleting file nr: 5 with id: 2303.00248 - no '.tex' file found...
 ===== Handling file nr: 6 with id: 2303.00224 =====
 ===== Handling file nr: 7 with id: 2303.00640 =====
 ===== Handling file nr: 8 with id: 2303.00444 =====
 ===== Handling file nr: 9 with id: 2303.00072 =====
 ===== Handling file nr: 10 with id: 2303.00416 =====
 Deleting file nr: 10 with id: 2303.00416 - no '.tex' file found...
 ===== Handling file nr: 11 with id: 2303.00081 =====
 ===== Handling file nr: 12 with id: 2303.00648 =====
 ===== Handling file nr: 13 with id: 2303.00679 =====
 ===== Handling file nr: 14 with id: 2303.00225 =====
 ===== Handling file nr: 15 with id: 2303.00469 ===

In [127]:
file_names = [download_directory+file_name for file_name in os.listdir(download_directory) if file_name != ".DS_Store"]
output_file_name = "equations.txt"
math_scraper(input_names=file_names,output_name=output_file_name)